# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:\Anaconda\python-api-challenge\weatherpy\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount isa,-20.7333,139.5000,17.87,34,0,3.60,AU,1683845111
1,1,jamestown,42.0970,-79.2353,23.80,31,0,4.63,US,1683845111
2,2,waitangi,-43.9535,-176.5597,11.57,79,38,0.89,NZ,1683845111
3,3,adamstown,-25.0660,-130.1015,21.72,57,73,7.85,PN,1683845112
4,4,camacupa,-12.0167,17.4833,13.49,56,2,0.30,AO,1683845112


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
# Configure the map plot
earth_plot=city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    titles = 'Humidity per cities',
    size = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    color= 'City',
    tiles= 'OSM'
)
import json
# Display the map
earth_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_data_df= city_data_df.loc[(city_data_df['Max Temp']<= 27) & (city_data_df['Max Temp']>= 21) & (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness']== 0)]

# Drop any rows with null values
narrow_data_df.dropna()

# Display sample data
narrow_data_df.head

<bound method NDFrame.head of      City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
46        46     el arahal  37.2627   -5.5453     21.41        59           0   
91        91      wahpeton  46.2652  -96.6059     26.08        60           0   
180      180     carnarvon -24.8667  113.6333     21.59        54           0   
183      183          idri  27.5000   13.2667     21.61        34           0   
186      186      show low  34.2542 -110.0298     21.67        16           0   
237      237  porto seguro -16.4497  -39.0647     25.29        83           0   
239      239     ludington  43.9553  -86.4526     26.01        30           0   
243      243        laguna  38.4210 -121.4238     26.17        38           0   
280      280  saint-pierre -21.3393   55.4781     22.82        68           0   
454      454   tamanrasset  22.7850    5.5228     25.95        14           0   
474      474  apple valley  34.5008 -117.1859     26.17        20           0  

### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df_clean=hotel_df.drop(['Max Temp','Cloudiness','Wind Speed','Date'], axis=1)
hotel_df_clean['Hotel Name']= ''
# Display sample data
hotel_df_clean

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
0,0,mount isa,-20.7333,139.5000,34,AU,
1,1,jamestown,42.0970,-79.2353,31,US,
2,2,waitangi,-43.9535,-176.5597,79,NZ,
3,3,adamstown,-25.0660,-130.1015,57,PN,
4,4,camacupa,-12.0167,17.4833,56,AO,
...,...,...,...,...,...,...,...
577,577,mariveles,14.4833,120.4833,83,PH,
578,578,mancora,-4.1078,-81.0475,68,PE,
579,579,lishui,28.4604,119.9103,91,CN,
580,580,taltal,-25.4000,-70.4833,82,CL,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': 'accomodation.hotel',
    'limit' : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude}, {latitude}, {radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount isa - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
camacupa - nearest hotel: No hotel found
catalhoyuk - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
keflavik - nearest hotel: No hotel found
concepcion - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
xunchang - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
skeldon - nearest hotel: No hotel found
opelousas - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
kolwezi - nearest hotel: No hotel found
hobyo - nearest hotel: No hotel fo

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)